Semi-supervised Community Detection using Graph Convolutional Network
=====================

Predicting community memberships of a network of entities is a common task in many real-world scenarios
working with graph data. In this tutorial, we demonstrate how to implement a Graph Convolutional Network (GCN)
[Kipf & Welling](https://arxiv.org/abs/1609.02907) using DGL to solve one such community detection problem in
a semi-supervised setting.

More specifically, you will learn:
- How to load graph data to DGLGraph
- How to manipulate node/edge features on the graph
- How to write a Graph Convolutional Layer using message passing APIs
- Train the model and visualize the result.

In [ ]:
# A bit of setup, just ignore this cell
import matplotlib.pyplot as plt

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['animation.html'] = 'html5'

Zachery's Karate Club
---
We start by creating the well-known *"Zachary's karate club"* social network. The network captures 34 members of a karate club, documenting pairwise links between members who interacted outside the club. The club later splits into two communities led by the instructor (node 0) and club president (node 33). You could read more about the story in the [wiki page](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) A visualization of the network and the community is as follows:

![karate](https://www.dropbox.com/s/uqzor4lqsmbnz8k/karate1.jpg?dl=1)

Load graph data
---
Let's see how we can load such graph to DGL. We start with importing `dgl` and other relevant packages.

In [ ]:
import os
os.environ['DGLBACKEND'] = 'mxnet'

import dgl
# Load MXNet as backend
dgl.load_backend('mxnet')

DGL is platform-agnostic and can support multiple popular tensor-based frameworks such as [PyTorch](https://pytorch.org) and [MXNet](https://mxnet.apache.org/). In this session, we use MXNet/Gluon backend and provide equivalent Pytorch-based implementation in the comments.

To create a graph in dgl, use `g = dgl.DGLGraph(graph_data)`. We support a wide range of `graph_data`. Here are some examples:

* An edge list (e.g. `[(0, 1), (1, 2), ...]`)
* A [`networkx`](https://networkx.github.io/) graph object.
* A scipy sparse matrix.

Since `networkx` already provides an API to create a karate club network, we can create a DGLGraph from it.

In [ ]:
import networkx as nx

G = dgl.DGLGraph(nx.karate_club_graph())

Define a GCN layer using message passing paradigm
---

Graph convolutional network (GCN) is a popular model proposed by [Kipf & Welling](https://arxiv.org/abs/1609.02907) to encode graph structure. The model consists of several layers, each perform convolution-like operation defined on graph:
$$
Y=\hat{A}XW
$$

, where $X$ is the node embedding tensor (stacked along the first dimension), $W$ is a projection matrix (the weight parameter) and $\hat{A}$ is the normalized adjacency matrix:
$$
\hat{A} = D^{-\frac{1}{2}}AD^{-\frac{1}{2}}
$$

The equations above involve a matrix multiplication between the normalized adjacency matrix and node features. And this can be expressed in terms of **message passing paradigm**:
* message phase: all nodes first compute and send out messages along out-going edges.
* reduce phase: all node then collect in-coming messages, aggregate them and update their own embedding.

In [ ]:
import mxnet.gluon as gluon
import mxnet.gluon.nn as nn

# Define the GraphConv module
class GraphConv(gluon.Block):
    def __init__(self, in_feats, out_feats):
        super(GraphConv, self).__init__()
        self.linear = nn.Dense(out_feats)
    
    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first perform linear transformation
        h = self.linear(inputs)
        # set the node features
        g.ndata['h'] = h
        # trigger message passing, using the message_func and reduce_func.
        g.update_all(message_func, reduce_func)
        # get the result node features
        return g.ndata.pop('h')

In [ ]:
# >>> For torch users
#
# import torch.nn as nn
# import torch.nn.functional as F
#
# # Define the GraphConv module
# class GraphConv(nn.Module):
#     def __init__(self, in_feats, out_feats):
#         super(GraphConv, self).__init__()
#         self.linear = nn.Linear(in_feats, out_feats)
#    
#     def forward(self, g, inputs):
#         # g is the graph and the inputs is the input node features
#         # first perform linear transformation
#         h = self.linear(inputs)
#         # set the node features
#         g.ndata['h'] = h
#         # trigger message passing, using the defined message_func and reduce_func.
#         g.update_all(message_func, reduce_func)
#         # get the result node features
#         h = g.ndata.pop('h')
#         return h
# <<<

Now let's see how we define `message_func` and `reduce_func` in DGL:

Suppose the current embedding of node $i$ after the linear transformation (i.e. multiplying weight matrix $W$) is $h_i$. From the equation of GCN above, each node sends out the embedding after linear transformation to their neighbors. Then the message from node $j$ to node $i$ can be computed as
$$m_{j\rightarrow i} = h_j$$

So the message function takes out node feature `h` as the message, and can be defined using DGL's built-in functions:

In [ ]:
import dgl.function as fn
message_func = fn.copy_u('h', 'm')

Each node aggregates received messages by summation. And for simplicity, we first ignore the normalization. So the aggregated messages on node $i$ can be computed as
$$\tilde{h_i} = \sum\limits_{j\in \mathcal{N}(i)}m_{j\rightarrow i}$$, where $\mathcal{N}(j)$ is the neighbor set of node $i$.

In [ ]:
reduce_func = fn.sum('m', 'h')

We then define a two-layer Graph Convolutional Network using the above module.

In [ ]:
# Define a 2-layer GCN model
class GCN(gluon.Block):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, num_classes)
    
    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = nd.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
# >>> For torch users
# 
# class GCN(nn.Module):
#     def __init__(self, in_feats, hidden_size, num_classes):
#         super(GCN, self).__init__()
#         self.gcn1 = GraphConv(in_feats, hidden_size)
#         self.gcn2 = GraphConv(hidden_size, num_classes)
#     
#     def forward(self, g, inputs):
#         h = self.gcn1(g, inputs)
#         h = torch.relu(h)
#         h = self.gcn2(g, h)
#         return h
# <<<

Now let's train this model to predict the club membership after the split. To train the model, we adopt Kipf's semi-supervised setting:
* Only the instructor node (node 0) and the president node (node 33) are labeled.
* The initial node feature is a one-hot encoding of the node id.

In [ ]:
from mxnet import nd

inputs = nd.eye(34)  # featureless inputs
labeled_nodes = nd.array([0, 33])  # only the instructor and the president nodes are labeled
labels = nd.array([0, 1])  # their labels are different

In [ ]:
from mxnet import autograd
net = GCN(34, 5, 2)
net.initialize()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.01})
loss_fn = gluon.loss.SoftmaxCELoss()

all_logits = []
for epoch in range(30):
    with autograd.record():
        logits = net(G, inputs)
        # we only compute loss for node 0 and node 33
        loss = loss_fn(logits[labeled_nodes], labels).sum()
    all_logits.append(logits.detach())
    
    loss.backward()
    trainer.step(batch_size=1)
    
    print('Epoch %d | Loss: %.4f' % (epoch, loss.asscalar()))

In [ ]:
# >>> For torch users
# inputs = torch.eye(34)  # featureless inputs
# labeled_nodes = torch.tensor([0, 33])  # only the instructor and the president nodes are labeled
# labels = torch.tensor([0, 1])  # their labels are different
# net = GCN(34, 5, 2)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
# 
# all_logits = []
# for epoch in range(30):
#     logits = net(G, inputs)
#     all_logits.append(logits.detach())
#     logp = F.log_softmax(logits, 1)
#     # we only compute loss for node 0 and node 33
#     loss = F.nll_loss(logp[labeled_nodes], labels)
# 
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
# 
#     print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))
# <<<

Now let's visualize the results. Since the final node embedding is a vector of length two (for predicting two classes), we can plot it as a point on a 2D plot and visualize how the final embeddings cluster towards each other.

In [ ]:
# Visualize the node classification using the logits output.
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML

fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()
nx_G = G.to_networkx()
def draw(i):
    cls1color = '#00FFFF'
    cls2color = '#FF00FF'
    pos = {}
    colors = []
    for v in range(34):
        pos[v] = all_logits[i][v].asnumpy()
        cls = np.argmax(pos[v])
        colors.append(cls1color if cls else cls2color)
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    nx.draw(nx_G.to_undirected(), pos, node_color=colors, with_labels=True, node_size=500)

ani = animation.FuncAnimation(fig, draw, frames=len(all_logits), interval=200)
HTML(ani.to_html5_video())